In [1]:
import pymc3 as pm    
import theano.tensor as tt
from theano.compile.ops import as_op
import numpy as np
import scipy.stats as stats

X = []
X.extend(stats.multinomial.rvs(n=10, p=[0.1,0.2,0.3,0.4,0.0], size=100))
X.extend(stats.multinomial.rvs(n=10, p=[0.1,0.2,0.2,0.3,0.2], size=25))
X = np.array(X)

In [3]:
X.shape

(125, 5)

In [ ]:
@as_op(itypes=[tt.lscalar, tt.dvector, tt.dvector], otypes=[tt.dvector])
def mswitch(tau, p1, p2):
    out = np.empty((len(X),5))
    out[:tau] = p1.reshape(1,-1)
    out[tau:] = p2.reshape(1,-1)
    return out

In [7]:
def mswitch(X, tau, p1, p2):
    p = tt.zeros(X.shape)
    p = tt.inc_subtensor(p[tt.arange(X.shape[0])<tau, ...], p1)
    p = tt.inc_subtensor(p[tt.arange(X.shape[0])>=tau, ...], p2)
    return p

In [8]:
with pm.Model() as model:
    p1 = pm.Dirichlet('p1', np.ones(5))
    p2 = pm.Dirichlet('p2', np.ones(5))
    tau = pm.Uniform('tau', 0, X.shape[0])
    p = mswitch(X, tau, p1, p2)
    X_ = pm.Multinomial('X_',10, p, observed=X)

    trace = pm.sample()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [tau, p2, p1]
Sampling 2 chains: 100%|██████████| 2000/2000 [05:35<00:00,  3.09draws/s]
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The estimated number of effective samples is smaller than 200 for some parameters.
